<a href="https://colab.research.google.com/github/cyFou/testColab/blob/main/tinyllm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source du tuto
https://levelup.gitconnected.com/building-a-perfect-million-parameter-llm-like-chatgpt-in-python-3b16e26b4139


# Creation du fichier jeu de données

In [1]:
# faire un ctrl C en fonction de la taille souhaité du jdd
#!wget https://www.modelscope.cn/datasets/deepctrl/deepctrl-sft-data/resolve/master/sft_data_en.jsonl

import os,urllib
url = 'https://www.modelscope.cn/datasets/deepctrl/deepctrl-sft-data/resolve/master/sft_data_en.jsonl'
filename = './sft_data_en.jsonl'
if not os.path.isfile(filename):
    urllib.request.urlretrieve(url, filename)


KeyboardInterrupt: 

In [ ]:
# Supprimer la dernière ligne du fichier sft_data_en.jsonl car mal formaté du fait du Ctrl C
def remove_last_line(filepath):
    """Removes the last line of a large file efficiently.

    Args:
      filepath: The path to the file.
    """
    try:
        with open(filepath, 'rb+') as f:
            f.seek(-2, os.SEEK_END)  # Go to the second-to-last byte
            while f.read(1) != b'\n':
                f.seek(-2, os.SEEK_CUR)  # Back up two bytes
                if f.tell() == 0:
                    # Handle the case where there's only one line or no newline chars.
                    f.truncate(0)
                    return
            f.truncate()
    except OSError as e:
        print(f"Error removing last line from {filepath}: {e}")
        return

remove_last_line("sft_data_en.jsonl")

In [2]:
# Supprimer la dernière ligne du fichier sft_data_en.jsonl car mal formaté du fait du Ctrl C
!sed -i '$d' sft_data_en.jsonl


In [5]:
# import json
# from tqdm import tqdm

# file_path = "sft_data_en.jsonl" # Downloaded filepath (23 GB RAM REQUIRED!!!!!!!!!!!!!!!!!!!!!)

# # Read the JSONL file with tqdm progress bar
# data = []
# with open(file_path, "r", encoding="utf-8") as file:
#     # Iterate through each line of the JSONL file
#     for line in tqdm(file, desc="Loading JSONL file"):
#         # Parse each line as a JSON object and append to data
#         data.append(json.loads(line))

# print(len(data))

Loading JSONL file: 29218it [00:01, 16275.08it/s]

29218


In [3]:

##### retravaille le fichier de données : ligne de 512 caratère, supression (en grande partie) des lignes avec caratère asiatique

import json
from tqdm import tqdm
import re

file_path = "sft_data_en.jsonl"

# Define the output file name
output_file = "petrain_data.jsonl"

lenAfterFilter = 0
# Write data to JSONL format
asian_char_pattern = re.compile(r'[\u4E00-\u9FFF\u3040-\u30FF\uAC00-\uD7AF]')
with open(file_path, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in tqdm(infile, desc="Filter JSONL file"):
        item = json.loads(line)
        io_length = len(item['input']) + len(item['output'])
        if io_length < 512 and not asian_char_pattern.search(line):
            json.dump({"text": f"{item['input']}\n{item['output']}"}, outfile, ensure_ascii=False)
            outfile.write("\n")  # Newline for each JSONL entry
            lenAfterFilter+=1
print(f"New JSONL file saved as {output_file}")
print(lenAfterFilter)

Filter JSONL file: 6849it [00:00, 7046.28it/s]

New JSONL file saved as petrain_data.jsonl
275


# Creation du dataset

In [5]:
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import json
from tqdm import tqdm


In [2]:
class PretrainDataset(Dataset):
    """Dataset for pretraining."""

    def __init__(self, data_path, tokenizer, max_length=512):
        """
        Initializes the PretrainDataset.

        Args:
            data_path (str): Path to the JSONL data file.
            tokenizer: The tokenizer to use.
            max_length (int): Maximum sequence length.
        """
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(data_path)

    def load_data(self, path):
        """
        Loads data from a JSONL file.

        Args:
            path (str): Path to the JSONL file.

        Returns:
            list: A list of samples loaded from the file.
        """
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            for line_num, line in enumerate(f, 1):  # enumerate starts at 1 for line_num
                data = json.loads(line.strip())
                samples.append(data)
        return samples

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.samples)

    def __getitem__(self, index):
        """
        Retrieves a sample from the dataset.

        Args:
            index (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple containing the input (X), target (Y), and loss mask.
        """
        sample = self.samples[index]

        # Construct the input text, including BOS and EOS tokens.
        text = f"{self.tokenizer.bos_token}{str(sample['text'])}{self.tokenizer.eos_token}"
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',  # Pad to max_length
            truncation=True,  # Truncate to max_length
            return_tensors='pt'  # Return PyTorch tensors
        )
        input_ids = encoding.input_ids.squeeze()  # Remove extra dimension
        loss_mask = (input_ids != self.tokenizer.pad_token_id)  # Create loss mask (ignore padding)

        # Create input (X) and target (Y) tensors, shifting by one position.
        X = torch.tensor(input_ids[:-1], dtype=torch.long)
        Y = torch.tensor(input_ids[1:], dtype=torch.long)
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)  # Shift loss mask as well
        return X, Y, loss_mask

In [8]:
print(tokenizer.bos_token)

<|endoftext|>


In [3]:
# Configuration for our Transformer model
model_config = {
    "vocab_size": 6400,       # Size of the vocabulary
    "dim": 512,               # Dimensionality of the embeddings and hidden states
    "n_heads": 8,             # Number of attention heads
    "n_kv_heads": 2,          # Number of key-value heads (as specified in the LMConfig)
    "norm_eps": 1e-5,         # Epsilon for RMSNorm
    "dropout": 0.0,           # Dropout probability
    "max_seq_len": 1024,      # Maximum sequence length
    "rope_theta": 10000.0,    # Theta parameter for RoPE
    "multiple_of": 64,        # Used for hidden dimension calculation in FFN
    "hidden_dim": None,       # Hidden dimension of the FFN (calculated if None)
    "n_layers": 8,            # Number of Transformer blocks
    "flash_attn": True,       # Use flash attention if available
}


In [6]:

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Create the training dataset
train_ds = PretrainDataset("petrain_data.jsonl", tokenizer, max_length=model_config["max_seq_len"])

# Create the data loader
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, drop_last=False)


In [ ]:
x,y,mask = next(iter(train_ds))
print(x.shape)
print(y.shape)
print(mask.shape)
print("x=>",x)
print("y=>",y)
print("x=>",tokenizer.decode(x))
print("y=>",tokenizer.decode(y))


In [13]:
 # Construct the input text, including BOS and EOS tokens.
text = f"{tokenizer.bos_token}{'Bonjour'}{tokenizer.eos_token}"
encoding = tokenizer(
    text,
    max_length=10,
    padding='max_length',  # Pad to max_length
    truncation=True,  # Truncate to max_length
    return_tensors='pt'  # Return PyTorch tensors
)
input_ids = encoding.input_ids.squeeze()  # Remove extra dimension
loss_mask = (input_ids != tokenizer.pad_token_id)  # Create loss mask (ignore padding)
print(input_ids)
print(loss_mask)

# Create input (X) and target (Y) tensors, shifting by one position.
X = torch.tensor(input_ids[:-1], dtype=torch.long)
Y = torch.tensor(input_ids[1:], dtype=torch.long)
loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)  # Shift loss mask as well

print(X)
print(Y)
print(loss_mask)

tensor([50256, 20682,    73,   454, 50256, 50257, 50257, 50257, 50257, 50257])
tensor([ True,  True,  True,  True,  True, False, False, False, False, False])
tensor([50256, 20682,    73,   454, 50256, 50257, 50257, 50257, 50257])
tensor([20682,    73,   454, 50256, 50257, 50257, 50257, 50257, 50257])
tensor([1, 1, 1, 1, 0, 0, 0, 0, 0])


C:\Users\fcyri\AppData\Local\Temp\ipykernel_1420\2342458088.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(input_ids[:-1], dtype=torch.long)
C:\Users\fcyri\AppData\Local\Temp\ipykernel_1420\2342458088.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(input_ids[1:], dtype=torch.long)
C:\Users\fcyri\AppData\Local\Temp\ipykernel_1420\2342458088.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long)  # Shift loss mask as well


# Creation du model

In [ ]:
class RMSNorm(torch.nn.Module):
    def __init__(self, dim: int, eps: float):
        super().__init__()
        self.eps = eps  # Small constant for numerical stability
        self.weight = nn.Parameter(torch.ones(dim)) # Learnable scaling parameter

    def forward(self, x):
        # Calculate the root mean square (RMS) and normalize
        return self.weight * (x.float() * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)).type_as(x)

NameError: name 'torch' is not defined